# Stock Trading Scheme

#### Importing packages

In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
import quandl
from stockAnalysisTools import numToString

#### The cell below will be used to pull all the stock data we want and make a dictionary for accessing it

In [2]:
quandl.ApiConfig.api_key = "d1JcayaRuvKSaq7GaEHs"
aapl = "aapl"
goog  = "goog"
stockList = [aapl, goog]
finData = dict()

for ticker in stockList:
    finData[ticker] = quandl.get("WIKI/"+ticker)["Adj. Close"]

#### Bollinger bands and Rolling Mean Calculated

#### Creating object for stock data

In [5]:
#Not using this class yet
class Stock(object):
    
    def __init__(self, name):
        self.name = ""
        self.num = 0

#### Creating an object to hold portfolio data

In [6]:
class Portfolio(object):
    names = list()
    stocks = list()
    uninvestedMoney = 0
    
    def addStock(self, name):
        
        if(name not in finData.keys()):
            print("We haven't pulled up data for this stock, so it can't be added")
            return

#         self.stockList.append(Stock(name))
        self.names.append(name)
        self.stocks.append(0)
    
    def addMoney(self, money):
        
        self.uninvestedMoney += money
        
    def reset(self):
        self.uninvestedMoney = 0
        for i in range(len(self.stocks)):
            self.stocks[i] = 0
#             self.stockList[i].num = 0
            
    def buyStock(self, name, date = 0, price = 0, num = 0, noLimit=True):
        if name not in self.names:
            print("Stock you want to buy isn't in the self")
            print("It must be added to the portfolio before you can buy")
            return

        if(date == 0):
            price_of_stock = price
        else:
            price_of_stock = finData[name].loc[date+" 00:00:00"]

        if(noLimit):
            numOfStock = self.uninvestedMoney/price_of_stock
            self.uninvestedMoney -= numOfStock * price_of_stock
            self.stocks[self.names.index(name)] += numOfStock
        else:
            if(num * price_of_stock() > self.uninvestedMoney):
                print("Can't buy this much stock")
            else:
                self.uninvestedMoney -= num * price_of_stock
                self.stocks[self.names.index(name)] += num 
                
    def sellStock(self, name, date = 0, price = 0, num = 0, noLimit=True):
        if name not in self.names:
            print("Stock you want to buy isn't in the self")
            print("It must be added to the portfolio before you can buy")
            return

        if(date == 0):
            price_of_stock = price
        else:
            price_of_stock = finData[name].loc[date+" 00:00:00"]

        numOfStock = self.stocks[self.names.index(name)]

        if(noLimit):
            self.stocks[self.names.index(name)] = 0

            self.uninvestedMoney += numOfStock * price_of_stock
        else:
            if(numOfStock - num < 0):
                print("Can't sell this much stock")
            else:
                self.uninvestedMoney += num * price_of_stock
                self.stocks[self.names.index(name)] -= num
        

#### Creating Portfolio

In [7]:
my_portfolio = Portfolio()
my_portfolio.addStock(aapl)

#### Simple Trading Strategy per 30 days

In [93]:
startDate = '2001-07-01'
endDate = '2018-07-01'

aaplReturn = finData[aapl].pct_change().shift(-1).dropna()
aaplSharpe = aaplReturn.mean()/aaplReturn.std()
aaplRollingStat = finData[aapl].loc[startDate: endDate].rolling(window=30,center=False).mean().dropna()
aaplRollingStd = finData[aapl].loc[startDate: endDate].rolling(window=30,center=False).std().dropna()

aaplRollingStat = aaplRollingStat.reset_index()

aaplRollingStat = aaplRollingStat.rename(index=str, columns={"Adj. Close": "Rolling Mean"})
aaplRollingStat["Std"] = aaplRollingStd.values
aaplRollingStat["Upper Band"] = aaplRollingStat.apply(lambda row: row["Rolling Mean"] + 2*row["Std"], axis=1)
aaplRollingStat["Lower Band"] = aaplRollingStat.apply(lambda row: row["Rolling Mean"] - 2*row["Std"], axis=1)

aaplRollingStat = aaplRollingStat.set_index("Date")

aaplRollingStat["Adj. Close"] = finData[aapl].loc[aaplRollingStat.index[0]:aaplRollingStat.index[-1]].values

In [9]:
startDate = '2001-07-01'
endDate = '2018-07-01'

googReturn = finData[goog].pct_change().shift(-1).dropna()
googSharpe = googReturn.mean()/googReturn.std()
googRollingStat = finData[goog].loc[startDate: endDate].rolling(window=30,center=False).mean().dropna()
googRollingStd = finData[goog].loc[startDate: endDate].rolling(window=30,center=False).std().dropna()

googRollingStat = googRollingStat.reset_index()

googRollingStat = googRollingStat.rename(index=str, columns={"Adj. Close": "Rolling Mean"})
googRollingStat["Std"] = googRollingStd.values
googRollingStat["Upper Band"] = googRollingStat.apply(lambda row: row["Rolling Mean"] + 2*row["Std"], axis=1)
googRollingStat["Lower Band"] = googRollingStat.apply(lambda row: row["Rolling Mean"] - 2*row["Std"], axis=1)

googRollingStat = googRollingStat.set_index("Date")

googRollingStat["Adj. Close"] = finData[goog].loc[googRollingStat.index[0]:googRollingStat.index[-1]].values

In [100]:
#portfolio reset
my_portfolio.reset()
my_portfolio.addMoney(100)

# aaplRollingStat = aaplRollingStat.reset_index()

# aaplRollingStat["Buy Day"] = aaplRollingStat.apply(lambda row: 1 if row["Adj. Close"] < row["Lower Band"] else 0, axis=1)
# aaplRollingStat["Sell Day"] = aaplRollingStat.apply(lambda row: 1 if row["Adj. Close"] > row["Upper Band"] else 0, axis=1)
    

# aaplRollingStat = aaplRollingStat.set_index("Date")

# test = aaplRollingStat[["Buy Day", "Sell Day"]]
# test["test"] = test.apply(lambda row: row[0], axis=0)
# test
# # aaplRollingStat

# #Loop implementation gives a strange return value
# canBuy = True
# canSell = False
# for i, row in aaplRollingStat.iterrows():
#     if row["Adj. Close"] < row["Lower Band"] and canBuy:
#         my_portfolio.buyStock(aapl, price = row["Adj. Close"])
#         canBuy = False
#         canSell = True
#     elif row["Adj. Close"] > row["Upper Band"] and canSell:
#         my_portfolio.sellStock(aapl, price = row["Adj. Close"])
#         canBuy = True
#         canSell = False
# #         print(i)
        
# if 0 not in my_portfolio.stocks:
#     my_portfolio.sellStock(aapl, price = finData[aapl][-1])



canBuy = True
canSell = False
buy = list()
sell = list()
for i, row in aaplRollingStat.iterrows():
#     print(aaplRollingStat.loc[i])
    if row["Adj. Close"] < row["Lower Band"] and canBuy:
        buyVal = 1
        canBuy = False
        canSell = True
    else:
        buyVal = 0
        
    if row["Adj. Close"] > row["Upper Band"] and canSell:
        sellVal = 1
        canBuy = True
        canSell = False
    else:
        sellVal = 0
        
    buy.append(buyVal)
    sell.append(sellVal)
    
return_df = pd.DataFrame()
return_df["Buy Day"] = pd.Series(buy, index = aaplRollingStat.index)
return_df["Sell Day"] = pd.Series(sell, index = aaplRollingStat.index)
return_df["Adj. Close"] = aaplRollingStat["Adj. Close"]

return_df["Returns"] = (return_df["Buy Day"]*return_df["Adj. Close"]) + (return_df["Sell Day"]*return_df["Adj. Close"])
return_df["Returns"] = return_df["Returns"].replace(0 , np.nan)
return_df["Returns"] = return_df["Returns"].pct_change(fill_method = "pad")
# return_df["Returns"] = return_df["Returns"].replace(np.nan, 0)
return_ser = return_df["Returns"].dropna()
# print(return_df["Returns"].dropna())
return_ser
# for i, row in return_df.iterrows():
# #     if np.notna(row["Returns"]) and (row["Returns"] != 0):
#     if not (np.isnan(row["Returns"])):
#         print(row["Returns"])
    

Date
2001-11-26    0.236690
2002-04-26    0.076743
2002-10-25   -0.329857
2002-12-19   -0.079118
2003-02-18    0.075352
2003-04-11   -0.135560
2003-05-05    0.218939
2003-11-14    0.333748
2004-01-05    0.033085
2005-04-14    2.361299
2005-05-24    0.065486
2006-03-21    0.556927
2006-07-26    0.033328
2007-08-15    0.877251
2007-09-25    0.277565
2008-01-16    0.042173
2008-03-18   -0.168003
2008-06-12    0.304472
2009-02-06   -0.424449
2009-03-09   -0.166566
2009-03-23    0.295392
2010-01-29    0.783977
2010-03-05    0.139991
2010-08-24    0.095821
2010-09-16    0.152711
2011-03-16    0.193224
2011-07-07    0.082391
2011-11-21    0.033063
2011-12-27    0.101677
2012-04-24    0.378201
2012-07-02    0.057543
2012-10-08    0.081642
2013-07-30   -0.277451
2013-09-16   -0.000490
2013-10-21    0.158273
2014-01-10    0.028192
2014-04-24    0.071694
2014-10-16    0.199361
2014-10-23    0.089030
2015-01-05    0.017928
2015-01-29    0.119059
2015-06-29    0.055820
2015-07-20    0.060548
2015-0

In [102]:
cumulative_returns = (1+return_ser).cumprod()
final_value = cumulative_returns[-1]*my_portfolio.uninvestedMoney

print("Value of Investment $"+ numToString(final_value))

Value of Investment $15,126.6198594536581368


#### Rate of Return Series Generated